In [1]:
import os
from tqdm import tqdm
import glob
from xml.etree import ElementTree as ET
import re
import difflib
from torchmetrics.text import CharErrorRate, WordErrorRate
import jiwer
import numpy as np
#import cv2
import matplotlib.pyplot as plt
import cv2
from shapely.affinity import scale
import random


In [45]:
BASE = "/home/efittsc1/projects/latin-transcription/"
TRANSCRIBUS_SOURCE = BASE + "transkribus_model_result/page"
GROUND_TRUTH_SOURCE = BASE + "source"

In [46]:
transcribus_files = glob.glob(os.path.join(TRANSCRIBUS_SOURCE, "*.xml"))
transcribus_files_dics = {}

for file in transcribus_files:
    _, original_filename = os.path.basename(file).split("_")
    transcribus_files_dics[original_filename] = file



In [47]:
def get_namespace(element):
        m = re.match('\{.*\}', element.tag)
        return m.group(0)[1:-1] if m else ''

In [48]:
def extract_lines_from_xml(xml_file):
    lines = []
    #print(f"Processing {xml_file}")
    tree = ET.parse(xml_file)
    ns = {"ns": get_namespace(tree.getroot())}
    ET.register_namespace('', ns['ns'])
    root = tree.getroot()
    try:
        for text_region in root.findall('.//ns:TextRegion', ns):
            for lineno, text_line in enumerate(text_region.findall('.//ns:TextLine', ns)):
                text = text_line.find('.//ns:TextEquiv', ns).find('.//ns:Unicode', ns).text
                if text is None:
                    continue
                baseline = text_line.find('ns:Baseline', ns).get('points')
                #print(baseline)
                baseline_list = np.array([p.split(",") for p in baseline.split(" ")], dtype=int)
                text = text.strip()
                text = text.replace(",", ".")
                lines.append((text, baseline_list))
    except Exception as e:
        print(f"Error processing {xml_file}: {e}")
    return lines


In [49]:
transformation = jiwer.Compose([
    jiwer.ToLowerCase(),
    jiwer.RemovePunctuation(),
    jiwer.RemoveMultipleSpaces(),
    jiwer.Strip(),
])

In [50]:
for filename in tqdm(glob.glob(GROUND_TRUTH_SOURCE + "/*.xml")):
    if os.path.basename(filename) not in transcribus_files_dics:
        print(f"{os.path.basename(filename)} not found in transcribus files")
    #assert os.path.basename(filename) in transcribus_files_dics, f"{os.path.basename(filename)} not found in transcribus files"

100%|██████████| 100/100 [00:00<00:00, 602629.89it/s]

JUST1-633m20.xml not found in transcribus files


In [51]:
def get_line_spacing(baselines):
    center_x = np.median([(l.T[0][0] + l.T[0][-1])/2 for l in baselines])
    
    center_ys = []
    for line in baselines:
        xs, ys = line.T
        center_ys.append(np.interp(center_x, xs, ys))

    med_spacing = np.median(np.diff(center_ys))
    return int(round(med_spacing))

In [52]:
def box_from_baseline(baseline_points, med_spacing, height):
        lower_spacing = int(round(0.23 * med_spacing))
        upper_spacing = int(round(0.77 * med_spacing))
    
        baseline_points_polygon = [[max(p[0], 0), min(height-1, p[1])+lower_spacing] for p in baseline_points]
        baseline_points_polygon += [[max(p[0], 0), max(0, p[1]-upper_spacing)] for p in baseline_points[::-1]]
        return np.array(baseline_points_polygon, dtype=np.int32)

In [53]:
from shapely.geometry import Polygon
from shapely.strtree import STRtree

In [54]:
dirs = [ BASE +"transkribus_model_result/images/matched_lines", "transkribus_model_result/images/unmatched_lines"]

for d in dirs:
    if os.path.exists(d):
        for f in os.listdir(d):
            os.remove(os.path.join(d, f))
        os.rmdir(d)
    os.makedirs(d)
    

In [55]:
def polygon_to_points(polygon):
    array= np.array(polygon.exterior.coords, dtype=np.int32)
    return array[:-1]

In [56]:
def scale_np_array_y(array, scale):
    array = array.astype(float)
    x_values = array.T[0]
    y_values = array.T[1]
    x_values_unique = np.unique(x_values)
    
    for x in x_values_unique:
        indices = np.where(x_values == x)
        midpoint = np.mean(y_values[indices])
        y_values[indices] = scale * (y_values[indices] - midpoint) + midpoint
    return np.array([x_values, y_values]).T.astype(int)


In [57]:
char_error_rate = CharErrorRate()
word_error_rate = WordErrorRate()

data = {}

for filename in tqdm(glob.glob(GROUND_TRUTH_SOURCE + "/*.xml")):
    if os.path.basename(filename) not in transcribus_files_dics:
        continue
    transkribus_file = transcribus_files_dics[os.path.basename(filename)]
    ground_truth_file = filename
    transkribus_lines = extract_lines_from_xml(transkribus_file)
    ground_truth_lines = extract_lines_from_xml(ground_truth_file)
    t_lines = [t[0] for t in transkribus_lines]
    gt_lines = [t[0] for t in ground_truth_lines]

    t_baselines = [t[1] for t in transkribus_lines]
    gt_baselines = [t[1] for t in ground_truth_lines]

    try:
        #med_spacing_t = get_line_spacing(t_baselines)
        med_spacing_gt = get_line_spacing(gt_baselines)
    except Exception as e:
        print(filename)
        print(f"transkribus_lines: {t_baselines}")
        print(f"ground_truth_lines: {gt_baselines}")
        print(f"Error processing {filename}: {e}")
        continue

    image_path = filename.replace(".xml", ".jpeg")
    if not os.path.exists(image_path):
        image_path = filename.replace(".xml", ".jpg")
    if not os.path.exists(image_path):
        image_path = filename.replace(".xml", ".png")
    if not os.path.exists(image_path):
        raise Exception(f"Image not found for {filename}")
    

    image = cv2.imread(image_path)
    height, width, _ = image.shape

    def get_bbx(np_array):
        x = np_array[:, 0]
        y = np_array[:, 1]
        return np.array([[np.min(x), np.min(y)], [np.max(x), np.max(y)]])
    
    t_bounding_boxes = [get_bbx(t[1]) for t in transkribus_lines]
    gt_bounding_boxes = [get_bbx(t[1]) for t in ground_truth_lines]

    t_polygons = [box_from_baseline(t[1], med_spacing_gt, height) for t in transkribus_lines]
    gt_polygons = [box_from_baseline(t[1], med_spacing_gt, height) for t in ground_truth_lines]

    shapely_t_polygons = [Polygon(p.tolist()) for p in t_polygons]
    shapely_gt_polygons = [Polygon(p.tolist()) for p in gt_polygons]

    gt_strtree = STRtree(shapely_gt_polygons)

    matched_polygons = []

    for i, t_polygon in enumerate(shapely_t_polygons):
        possible_matches = gt_strtree.query(t_polygon)
        scaled_t_polygon = Polygon(scale_np_array_y(polygon_to_points(t_polygon), 1.6).tolist())
        for match_idx in possible_matches:
            match = shapely_gt_polygons[match_idx]
            scaled_match = Polygon(scale_np_array_y(polygon_to_points(match), 1.6).tolist())
            intersection = t_polygon.intersection(scaled_match)
            intersection2 = scaled_t_polygon.intersection(match)
            if intersection.area > 0.9 * t_polygon.area and intersection2.area > 0.9 * match.area:
                matched_polygons.append((i, shapely_gt_polygons.index(match)))
            
    
    for i, j in matched_polygons:
        random_color = (np.random.randint(0, 255), np.random.randint(0, 255), np.random.randint(0, 255))
        cv2.polylines(image, [t_polygons[i]], isClosed=True, color=random_color, thickness=3)
        cv2.polylines(image, [gt_polygons[j]], isClosed=True, color=random_color, thickness=2)
    # save image to images/works
    filename_ = "images/matched_lines/" + os.path.basename(filename).split('.')[0] + ".png"
    cv2.imwrite(filename_, image)

    
    unmatched_polygons_t = [i for i in range(len(t_polygons)) if i not in [m[0] for m in matched_polygons]]
    unmatched_polygons_gt = [i for i in range(len(gt_polygons)) if i not in [m[1] for m in matched_polygons]]

    if len(unmatched_polygons_t + unmatched_polygons_gt) != 0:
        new_image = cv2.imread(image_path)
        print(f"Unmatched polygons for {filename}")
        for i in unmatched_polygons_t:
            cv2.polylines(new_image, [t_polygons[i]], isClosed=True, color=(0, 255, 0), thickness=3)
        for i in unmatched_polygons_gt:
            cv2.polylines(new_image, [gt_polygons[i]], isClosed=True, color=(255, 0, 0), thickness=2)
        
        stacked_image = np.vstack([image, new_image])
        filename_ = "images/unmatched_lines/" + os.path.basename(filename).split('.')[0] + ".png"
        cv2.imwrite(filename_, stacked_image)

    data[os.path.basename(filename)] = {}
    for index, (i, j) in enumerate(matched_polygons):
        gt = transformation(ground_truth_lines[j][0])
        hyp = transformation(transkribus_lines[i][0])

        char_error_rate.update(hyp, gt)
        word_error_rate.update(hyp, gt)

        data[os.path.basename(filename)][index] = {
            "ground_truth": ground_truth_lines[j][0],
            "hypothesis": transkribus_lines[i][0],
            "ground_truth_index": j,
            "hypothesis_index": i,
        }


  0%|          | 0/100 [00:00<?, ?it/s]

  1%|          | 1/100 [00:00<00:27,  3.66it/s]

Unmatched polygons for /home/efittsc1/projects/latin-transcription/source/JUST1-734m3d.xml


  2%|▏         | 2/100 [00:00<00:32,  3.01it/s]

Unmatched polygons for /home/efittsc1/projects/latin-transcription/source/JUST1-633m84d.xml


...

Unmatched polygons for /home/efittsc1/projects/latin-transcription/source/JUST1-734m3b.xml


 99%|█████████▉| 99/100 [00:18<00:00,  4.44it/s]

Unmatched polygons for /home/efittsc1/projects/latin-transcription/source/JUST1-633m9d.xml


100%|██████████| 100/100 [00:18<00:00,  5.28it/s]

Unmatched polygons for /home/efittsc1/projects/latin-transcription/source/JUST1-734m17da.xml


In [58]:
character_error_rate_ = float(char_error_rate.compute())
word_error_rate_ = float(word_error_rate.compute())


In [59]:
print(f"Character Error Rate: {character_error_rate_}")
print(f"Word Error Rate: {word_error_rate_}")

Character Error Rate: 0.22566388547420502
Word Error Rate: 0.5422852039337158


In [60]:
char_error_rate_old = CharErrorRate()
word_error_rate_old = WordErrorRate()

data_old = {}

for filename in tqdm(glob.glob(GROUND_TRUTH_SOURCE + "/*.xml")):
    if os.path.basename(filename) not in transcribus_files_dics:
        continue
    transkribus_file = transcribus_files_dics[os.path.basename(filename)]
    ground_truth_file = filename
    transkribus_lines = extract_lines_from_xml(transkribus_file)
    ground_truth_lines = extract_lines_from_xml(ground_truth_file)

    transkribus_lines = [t[0] for t in transkribus_lines]
    ground_truth_lines = [t[0] for t in ground_truth_lines]

    transkribus_text = " ".join(transkribus_lines)
    ground_truth_text = " ".join(ground_truth_lines)
    
    gt = transformation(ground_truth_text)
    hyp = transformation(transkribus_text)  

    char_error_rate_old.update(hyp, gt)
    word_error_rate_old.update(hyp, gt)

    data_old[os.path.basename(filename)] = {
        "ground_truth": ground_truth_text,
        "hypothesis": transkribus_text,
    }

  1%|          | 1/99 [00:02<04:09,  2.55s/it]


KeyboardInterrupt: 

In [ ]:
character_error_rate_old = float(char_error_rate_old.compute())
word_error_rate_old = float(word_error_rate_old.compute())
print(f"Character Error Rate: {character_error_rate_old}")
print(f"Word Error Rate: {word_error_rate_old}")

Character Error Rate: 0.2582835257053375
Word Error Rate: 0.566498339176178
